### Домашняя работа № 6
Был взят датасет, приведенный к материалам урока.

Прочитаем датасет

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("/home/vyacheslav/Рабочий стол/Data Scientist/IV ЧЕТВЕРТЬ/I. Машинное обучение в бизнесе/6 lesson/training_project_data.csv")
data.head(3)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,NEXT_MONTH_DEFAULT
0,150000.0,2,2,2,24,1,2,0,0,0,...,17853.0,34527.0,28277.0,0.0,2145.0,10000.0,22000.0,3000.0,5093.0,0
1,50000.0,2,3,1,46,3,3,3,3,2,...,30748.0,30162.0,30016.0,0.0,0.0,0.0,0.0,3100.0,798.0,1
2,150000.0,2,2,1,41,-1,-1,-1,-1,0,...,6784.0,6923.0,14116.0,0.0,0.0,6784.0,159.0,167.0,760.0,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LIMIT_BAL           10000 non-null  float64
 1   SEX                 10000 non-null  int64  
 2   EDUCATION           10000 non-null  int64  
 3   MARRIAGE            10000 non-null  int64  
 4   AGE                 10000 non-null  int64  
 5   PAY_1               10000 non-null  int64  
 6   PAY_2               10000 non-null  int64  
 7   PAY_3               10000 non-null  int64  
 8   PAY_4               10000 non-null  int64  
 9   PAY_5               10000 non-null  int64  
 10  PAY_6               10000 non-null  int64  
 11  BILL_AMT1           10000 non-null  float64
 12  BILL_AMT2           10000 non-null  float64
 13  BILL_AMT3           10000 non-null  float64
 14  BILL_AMT4           10000 non-null  float64
 15  BILL_AMT5           10000 non-null  float64
 16  BILL_

У нас есть 24 признака и 1 целевая переменная (бинарная) - нужно определить наступление дефолта по кредиту

In [3]:
print(data.shape)

(10000, 24)


Всего 10000 кредитов

Посмотрим на соотношение классов

In [4]:
data.iloc[:, -1].value_counts()

0    7805
1    2195
Name: NEXT_MONTH_DEFAULT, dtype: int64

В целом соотношение классов 1 к 4, не так уж и плохо. Далее построим новые признаки.

In [5]:
data['PAY_1'].unique()

array([ 1,  3, -1,  0,  2])

In [6]:
col1 = ['SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'NEXT_MONTH_DEFAULT']
col = list(set(data.columns) - set(col1))

In [7]:
def transform_data(data):
    for i1, col1 in enumerate(col):
#         data[col1 + '_log'] = np.log(data[col1] + 1.1)
        data[col1 + '_exp'] = np.exp(data[col1] + 1.1)
        
        for i2, col2 in enumerate(col):
            data['%s_%s_1' % (col1, col2)] = data[col1] - data[col2]
            data['%s_%s_2' % (col1, col2)] = data[col1] + data[col2]
            data['%s_%s_3' % (col1, col2)] = data[col1] / (data[col2] + 0.1)
            data['%s_%s_4' % (col1, col2)] = data[col1] * data[col2]
            data['%s_%s_5' % (col1, col2)] = data[col1] ** data[col2]
            
#             data['%s_%s_11' % (col1, col2)] = data[col1] - np.log(data[col2] + 1)
#             data['%s_%s_22' % (col1, col2)] = data[col1] + np.log(data[col2] + 1)
#             data['%s_%s_33' % (col1, col2)] = data[col1] / (np.log(data[col2] + 1) + 0.1)
#             data['%s_%s_44' % (col1, col2)] = data[col1] * np.log(data[col2] + 1)
#             data['%s_%s_55' % (col1, col2)] = data[col1] ** np.log(data[col2] + 1)
            
    return data

In [8]:
def transform_data1(data):
    for i1, col1 in enumerate(col):
        for i2, col2 in enumerate(col):
#             data[col1 + '-'] = data[col1] - data[col2]
            data[col1 + '+'] = data[col1] + data[col2]
#             data[col1 + '/'] = data[col1] / (data[col2] + 0.1)
            data[col1 + '*'] = data[col1] * data[col2]
#             data[col1 + '**'] = data[col1] ** data[col2]
            
            
    return data

In [9]:
# data_fi = transform_data1(data)

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (используем - случайный лес)

In [10]:
# data_fi

In [11]:
from sklearn.model_selection import train_test_split

# x_data = data_fi.iloc[:,:-1]
# y_data = data_fi.iloc[:,-1]

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [12]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

# from sklearn.linear_model import LogisticRegression  
# from sklearn.neighbors import KNeighborsClassifier 

model = RandomForestClassifier(random_state=42)

# model = xgb.XGBClassifier()

# model = LogisticRegression()

# model = KNeighborsClassifier(n_neighbors=5)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [13]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

f1_ = []
roc_ = []
recall_ = []
precision_ = []

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    f1_.append(f1)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    roc_.append(roc)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    recall_.append(rec)
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    precision_.append(prc)
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 45.60%
roc: 65.03%
recall: 36.74%
precision: 60.08%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [54]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.55 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1208/2195 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [55]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    8792
 1    1208
Name: class_test, dtype: int64


In [56]:
mod_data.head(10)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,NEXT_MONTH_DEFAULT,class_test
0,150000.0,2,2,2,24,1,2,0,0,0,...,34527.0,28277.0,0.0,2145.0,10000.0,22000.0,3000.0,5093.0,0,-1
1,50000.0,2,3,1,46,3,3,3,3,2,...,30162.0,30016.0,0.0,0.0,0.0,0.0,3100.0,798.0,1,1
2,150000.0,2,2,1,41,-1,-1,-1,-1,0,...,6923.0,14116.0,0.0,0.0,6784.0,159.0,167.0,760.0,0,-1
3,150000.0,2,2,2,35,0,0,0,0,0,...,48983.0,50136.0,3500.0,2000.0,20000.0,2000.0,2000.0,2000.0,0,-1
4,70000.0,2,1,1,35,1,2,2,2,2,...,45971.0,47573.0,0.0,6500.0,1800.0,0.0,2500.0,2000.0,0,-1
5,170000.0,2,1,2,27,-1,-1,-1,-1,-1,...,759.0,602.0,272.0,272.0,755.0,759.0,602.0,2581.0,0,-1
6,260000.0,2,1,1,29,0,0,0,-1,-1,...,0.0,0.0,3090.0,0.0,0.0,0.0,0.0,141516.0,0,-1
7,60000.0,2,2,2,26,0,0,0,0,0,...,-1073.0,-1073.0,1668.0,2000.0,1500.0,0.0,0.0,22000.0,0,-1
8,230000.0,2,1,2,28,-1,2,0,0,0,...,20109.0,13700.0,0.0,27446.0,1400.0,0.0,274.0,0.0,0,-1
9,50000.0,2,3,1,53,0,0,0,0,0,...,20218.0,20645.0,1770.0,2009.0,2593.0,1000.0,752.0,1017.0,1,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [57]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [58]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1208, 25) (1208, 25)


In [59]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

/home/vyacheslav/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:35:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 31.21%
roc: 68.09%
recall: 74.09%
precision: 19.77%


In [60]:
print(f1_)
print(roc_)
print(recall_)
print(precision_)

[0.455988455988456, 0.38612677259525024, 0.41943573667711603, 0.41994301994301997, 0.3694874851013111, 0.33806566104702745, 0.31208136938724884]
[0.6502814397866983, 0.6469685231238278, 0.6583787783491806, 0.6314385993465708, 0.6600894347936485, 0.6636675187278753, 0.6809035128449297]
[0.3674418604651163, 0.7248236048749198, 0.7474860335195531, 0.7141472868217055, 0.7110091743119266, 0.7121495327102804, 0.7408716136631331]
[0.6007604562737643, 0.2631578947368421, 0.29150326797385623, 0.2974172719935432, 0.249597423510467, 0.22164048865619546, 0.19767441860465115]


In [24]:
print(f'|   RESULTS   | PRECISION | RECALL | FSCORE | ROC_AUC |')
print(f'|   БЕЗ RNS   |    {precision_[0]:.2f}   |  {recall_[0]:.2f}  |  {f1_[0]:.2f}  |  {roc_[0]:.2f}   |')
print(f'|    С RNS    |    {precision_[1]:.2f}   |  {recall_[1]:.2f}  |  {f1_[1]:.2f}  |  {roc_[1]:.2f}   |')

|   RESULTS   | PRECISION | RECALL | FSCORE | ROC_AUC |
|   БЕЗ RNS   |    0.60   |  0.37  |  0.46  |  0.65   |
|    С RNS    |    0.26   |  0.72  |  0.39  |  0.65   |


### Вывод: большая разница между PRECISION и RECALL, что говорит о следующем:  В первом случае классификатор более пессимистичен и большему числу отбракованных верных ответов. Во втором случае классификатор более оптимистичен и более высокому числу ложно-положиетльных ответов.

Эксперимент в сторону уменьшения/увеличения размера Р

In [61]:
print(f'|   RESULTS   | PRECISION | RECALL | FSCORE | ROC_AUC |')
print(f'|   БЕЗ RNS   |    {precision_[0]:.2f}   |  {recall_[0]:.2f}  |  {f1_[0]:.2f}  |  {roc_[0]:.2f}   |')
print(f'|  С RNS  5%  |    {precision_[3]:.2f}   |  {recall_[3]:.2f}  |  {f1_[3]:.2f}  |  {roc_[3]:.2f}   |')
print(f'|  С RNS 15%  |    {precision_[2]:.2f}   |  {recall_[2]:.2f}  |  {f1_[2]:.2f}  |  {roc_[2]:.2f}   |')
print(f'|  С RNS 25%  |    {precision_[1]:.2f}   |  {recall_[1]:.2f}  |  {f1_[1]:.2f}  |  {roc_[1]:.2f}   |')
print(f'|  С RNS 35%  |    {precision_[4]:.2f}   |  {recall_[4]:.2f}  |  {f1_[4]:.2f}  |  {roc_[4]:.2f}   |')
print(f'|  С RNS 45%  |    {precision_[5]:.2f}   |  {recall_[5]:.2f}  |  {f1_[5]:.2f}  |  {roc_[5]:.2f}   |')
print(f'|  С RNS 55%  |    {precision_[6]:.2f}   |  {recall_[6]:.2f}  |  {f1_[6]:.2f}  |  {roc_[6]:.2f}   |')

|   RESULTS   | PRECISION | RECALL | FSCORE | ROC_AUC |
|   БЕЗ RNS   |    0.60   |  0.37  |  0.46  |  0.65   |
|  С RNS  5%  |    0.30   |  0.71  |  0.42  |  0.63   |
|  С RNS 15%  |    0.29   |  0.75  |  0.42  |  0.66   |
|  С RNS 25%  |    0.26   |  0.72  |  0.39  |  0.65   |
|  С RNS 35%  |    0.25   |  0.71  |  0.37  |  0.66   |
|  С RNS 45%  |    0.22   |  0.71  |  0.34  |  0.66   |
|  С RNS 55%  |    0.20   |  0.74  |  0.31  |  0.68   |


### Вывод: 1) с увеличением размера Р PRECISION уменьшается, RECALL увеличивается, FSCORE уменьшается и ROC_AUC остается практически неизменным. Это говорит о том, что с увеличением размера Р классификатор становится более оптимистичным с увеличением ложно-положительных ответов. Гармоническое среднее уменьшается и ROC_AUC практически не меняется. В результате проведенного эксперимента для данной выборки я выбираю размер Р в 15%.

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Ваш ответ здесь: